# Project AV

## Fraud Detection System Using Data Visualization and Machine Learning

Norbert Tomàs Escudero (242695) Marc Sirvent Ruiz (240198)


The goal of this project is to build an integrated solution for detecting and preventing fraudulent financial transactions.

Our project focuses on answering the following questions:
- What trends and patterns are evident in transaction data, and how do they relate to fraudulent activity? (difference in trends and patterns from transaction data (non-fraudulent) and fraudulent)  
- Which features (e.g., transaction amount, card type, country) are the most important indicators of fraud? (Using SHAP on our predictive model of fraud) 
- How can we present insights and predictions in a user-friendly manner for decision-making? 

The results of this project will assist stakeholders in identifying fraud patterns through data-driven decisions.

https://www.kaggle.com/datasets/computingvictor/transactions-fraud-datasets?select=train_fraud_labels.json

## Setup and data imports

In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [13]:
GLOBAL_PATH = './data/'
USERS_DATASET = "users_data.csv"
CARDS_DATASET = "cards_data.csv"
TRANSACTIONS_DATASET = "transactions_data.csv"
TRAIN_FRAUD_LABELS = "train_fraud_labels.json"
MCC_CODES_DATASET = "mcc_codes.json"

users_df = pd.read_csv(GLOBAL_PATH + USERS_DATASET)
cards_df = pd.read_csv(GLOBAL_PATH + CARDS_DATASET)
transactions_df = pd.read_csv(GLOBAL_PATH + TRANSACTIONS_DATASET)
train_fraud_labels_df = pd.read_json(GLOBAL_PATH + TRAIN_FRAUD_LABELS)
mcc_codes_df = pd.read_json(GLOBAL_PATH + MCC_CODES_DATASET, typ='series')

In [14]:
print("USERS_DATASET")
display(users_df.head())

print("\nCARDS_DATASET")
display(cards_df.head())

print("\nTRANSACTIONS_DATASET")
display(transactions_df.head())

print("\nTRAIN_FRAUD_LABELS")
display(train_fraud_labels_df.head())

print("\nMCC_CODES_DATASET")
display(mcc_codes_df.head())

USERS_DATASET


,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1



CARDS_DATASET


,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No



TRANSACTIONS_DATASET


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN



TRAIN_FRAUD_LABELS


,target
10649266,No
23410063,No
9316588,No
12478022,No
9558530,No



MCC_CODES_DATASET


5812           Eating Places and Restaurants
5541                        Service Stations
7996    Amusement Parks, Carnivals, Circuses
5411            Grocery Stores, Supermarkets
4784                   Tolls and Bridge Fees
dtype: object

In [15]:
# Save df just for development
cards_df_original = cards_df.copy()
mcc_codes_df_original = mcc_codes_df.copy()
train_fraud_labels_df_original = train_fraud_labels_df.copy()
transactions_df_original = transactions_df.copy()
users_df_original = users_df.copy()

In [100]:
""" column1 = transactions_df[transactions_df['id'] == 12478022]
column2 = train_fraud_labels_df[train_fraud_labels_df.index == 12478022]
display(column1)
display(column2) """

" column1 = transactions_df[transactions_df['id'] == 12478022]\ncolumn2 = train_fraud_labels_df[train_fraud_labels_df.index == 12478022]\ndisplay(column1)\ndisplay(column2) "

## Data Cleaning

In [16]:
#Auxiliary functions

def remove_dollar(df, columns_to_clean):
    for column in columns_to_clean:
        df[column] = df[column].str.replace(r'[\$,]', '', regex=True)
        df[column] = pd.to_numeric(df[column])
    return df

In [29]:
# Users df
users_df = users_df_original.copy()
users_df = pd.get_dummies(users_df, columns=['gender'], drop_first=True)
users_df = remove_dollar(users_df, ['per_capita_income', 'yearly_income', 'total_debt'])
users_df = users_df.drop(columns=['address'])

users_df.head()

,id,current_age,retirement_age,birth_year,birth_month,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,gender_Male
0,825,53,66,1966,11,34.15,-117.76,29278,59696,127613,787,5,False
1,1746,53,68,1966,12,40.76,-73.74,37891,77254,191349,701,5,False
2,1718,81,67,1938,11,34.02,-117.89,22681,33483,196,698,5,False
3,708,63,63,1957,1,40.71,-73.99,163145,249925,202328,722,4,False
4,1164,43,70,1976,9,37.76,-122.44,53797,109687,183855,675,1,True


In [22]:
# Cards df
cards_df = cards_df_original.copy()
cards_df = remove_dollar(cards_df, ['credit_limit'])

cards_df['expires'] = pd.to_datetime(cards_df['expires'], format="%m/%Y")
cards_df['acct_open_date'] = pd.to_datetime(cards_df['acct_open_date'], format="%m/%Y")
cards_df['year_pin_last_changed'] = pd.to_datetime(cards_df['year_pin_last_changed'], format="%Y")

cards_df = pd.get_dummies(cards_df, columns=['card_on_dark_web', 'has_chip', 'card_brand', 'card_type'], drop_first=True)

cards_df.head()

,id,client_id,card_number,expires,cvv,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,has_chip_YES,card_brand_Discover,card_brand_Mastercard,card_brand_Visa,card_type_Debit,card_type_Debit (Prepaid)
0,4524,825,4344676511950444,2022-12-01,623,2,24295,2002-09-01,2008-01-01,True,False,False,True,True,False
1,2731,825,4956965974959986,2020-12-01,393,2,21968,2014-04-01,2014-01-01,True,False,False,True,True,False
2,3701,825,4582313478255491,2024-02-01,719,2,46414,2003-07-01,2004-01-01,True,False,False,True,True,False
3,42,825,4879494103069057,2024-08-01,693,1,12400,2003-01-01,2012-01-01,False,False,False,True,False,False
4,4659,825,5722874738736011,2009-03-01,75,1,28,2008-09-01,2009-01-01,True,False,True,False,False,True


In [ ]:
# Transactions df
transactions_df = transactions_df_original.copy()

transactions_df = remove_dollar(transactions_df, ['amount'])
transactions_df['date'] = pd.to_datetime(transactions_df['date'], format="%Y-%m-%d %H:%M:%S")
transactions_df = transactions_df.drop(columns=['merchant_city', 'merchant_state', 'zip', 'errors']) # TODO: Dropped this columns because they have a lot of nulls
transactions_df = pd.get_dummies(transactions_df, columns=['use_chip'], drop_first=False)

transactions_df.head()

,id,date,client_id,card_id,amount,merchant_id,mcc,use_chip_Chip Transaction,use_chip_Online Transaction,use_chip_Swipe Transaction
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,59935,5499,False,False,True
1,7475328,2010-01-01 00:02:00,561,4575,14.57,67570,5311,False,False,True
2,7475329,2010-01-01 00:02:00,1129,102,80.00,27092,4829,False,False,True
3,7475331,2010-01-01 00:05:00,430,2860,200.00,27092,4829,False,False,True
4,7475332,2010-01-01 00:06:00,848,3915,46.41,13051,5813,False,False,True


In [30]:
# Train fraud labels df
train_fraud_labels_df = train_fraud_labels_df_original.copy()
train_fraud_labels_df = pd.get_dummies(train_fraud_labels_df, columns=['target'], drop_first=True)

train_fraud_labels_df.head()

,target_Yes
10649266,False
23410063,False
9316588,False
12478022,False
9558530,False


In [ ]:
# Ready to join the necessary datasets and columns to train, with the train_fraud_labels_df as target.

## Training

## Plots

In [ ]:
plt.figure(figsize=(10, 6))

sns.histplot(data=users_df[users_df["gender_encoded"]==1], x='yearly_income', bins=40, color='blue', alpha=0.3, label='Gender Male', stat='density')

sns.histplot(data=users_df[users_df["gender_encoded"] == 0], x='yearly_income', bins=40, color='red', alpha=0.3, label='Gender Female', stat='density')
plt.legend()

plt.title('Histogram for Gender')
plt.xlabel('Yearly Income')
plt.ylabel('Frequency')
plt.show()

In [ ]:
sns.histplot(data=users_df, x='current_age', bins=20, kde=True)
plt.title('Histogram with KDE')
plt.show()